1. $0^{th}$ level policy model $\pi^{(0)}(s_{t}|s_{t^{(1)}}^{(1)} \circ a_{t}^{(1)})$
2. $1^{th}$ level policy model $\pi^{(1)}()$

In [1]:
from model import sandwich_embedding as se 
import torch 
from model import GPTConfig

config = GPTConfig(vocab_size=128, n_layer=4, n_head=4, n_embd=64, K=3, L=2, device="cpu", _compile=False)
B, S, D = 3, 6, config.n_embd 
K = config.K
L = config.L

# (I). Sandwich embedding
# --------------------------------------------------------------------------------------------------------------------------
# Sandwich embedding ensemble (temporal predicted token embeddings ensemble)
S, S1, S2 = 18, 6, 2

tok_embed = torch.randn(B, S, D)
high_embed = torch.randn(B, S2, D)
low_embed = torch.randn(B, S1, D)

# test 1. (S, S1, S2) = (0, 1, 0) --> 1 (check)
# test 2. (S, S1, S2) = (3, 4, 1) --> 4 (check)
# test 3. (S, S1, S2) = (18, 6, 5) --> 19 (check)
cond_embed = se(low_embed, tok_embed, high_embed, K)


# (II). Conditional GPT
# --------------------------------------------------------------------------------------------------------------------------
from model import CondGPT, GPTConfig

condgpt = CondGPT(config)
idx = torch.randint(0, 128, (B, S))

condgpt.forward(idx, high_embed, low_embed)
condgpt.generate(idx, high_embed, low_embed)[1]

# (III). GAT 
# --------------------------------------------------------------------------------------------------------------------------
from model import GAT
gat = GAT(config)

idx = torch.randint(0, 128, (B, S))

# (a). one-step generation with GAT
gat.generate(idx)

[tensor([[ 64,   3,  49,  98,  14,  71,  73,  24,   6,  88,  97,  38,  60, 122,
          110,  10, 119,  93,   0],
         [112,  67,  71,  41,  35, 112,  19, 123,  57,  72,  74,  73,  95,  39,
           71,  56, 118,  63,   0],
         [ 91,  57,  48, 122,  71, 109,  95,  44,   9,  54,  15, 108,   1,  14,
            2,  59,  86, 124,   0]]),
 tensor([[0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0]])]